# **Coursera Capstone Project - Week 5**
### Author: Arvind Saigal

## **------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
### _**About the project**_    
#### 1. **The following script helps potential clients who are looking to buy suitable property in London**
#### 2. **With a pre-determined budged, this automation will recommend you locations and current average price of real estate where one can make a real estate investment**
#### 3. **Important amenities/ venues for each recommended location are displayed**
#### 4. **Facilities like elementary schools, high schools, hospitals & grocery stores are displayed for each recommended location**
## **------------------------------------------------------------------------------------------------------------------------------------------------------------------------**


In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

In [2]:
# Set the present working directory
os.chdir("/resources/data/Coursera")

#### **Source of dataset: http://landregistry.data.gov.uk/**

In [3]:
# Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("Price_Paid_Data.csv")

# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [4]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

#### **Select data only for city of LONDON**

In [5]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

#### **Calculate the street-wise average price of the property**

In [6]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

## **Input your Budget's Upper Limit and Lower Limit**

#### **Find the locations df_grp_price which fits your budget** 

#### **Please change the limits as per your budget**

In [7]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [8]:
# Display the dataframe
df_affordable

,Street,Avg_Price
45,ALBERT BRIDGE ROAD,2.383333e+06
299,BELSIZE LANE,2.350000e+06
538,BURNSALL STREET,2.286500e+06
553,BUSHNELL ROAD,2.200000e+06
675,CHALCOT GARDENS,2.200000e+06
773,CIRCUS ROAD WEST,2.448032e+06
903,CORNWALL TERRACE MEWS,2.350000e+06
913,COURT LANE GARDENS,2.360000e+06
1085,DORIA ROAD,2.325000e+06
1131,DULWICH WOOD AVENUE,2.297000e+06


#### **Read the street-wise coordinates into a dataframe**

In [9]:
london_data = pd.read_csv("Street Coordinates London.csv")

# Eliminate recurring word LONDON from individual names
london_data['Street'] = london_data['Street'].str.replace(', LONDON', '')

#### **Join the data to find the coordinates of locations which fit into client's budget**

In [19]:
df_pref_loc = pd.merge(london_data, df_affordable, on=['Street'], how='inner')

In [21]:
df_pref_loc.head(10)

,Unnamed: 0,Street,Latitude,Longitude,Avg_Price
0,77,DULWICH WOOD AVENUE,51.425586,-0.082416,2.297000e+06
1,251,SOUTH HILL PARK,51.557134,-0.164343,2.466667e+06
2,407,TEIGNMOUTH ROAD,51.550139,-0.214496,2.295000e+06
3,729,BURNSALL STREET,51.489042,-0.166883,2.286500e+06
4,948,FORDWYCH ROAD,51.551511,-0.206736,2.290000e+06
5,1044,PORTEN ROAD,51.498603,-0.214120,2.200000e+06
6,1058,ALBERT BRIDGE ROAD,51.477861,-0.164743,2.383333e+06
7,1137,EDITH VILLAS,51.491665,-0.206556,2.402500e+06
8,1322,WESTBOURNE GROVE,51.514797,-0.197071,2.300000e+06
9,1336,LADBROKE ROAD,51.508776,-0.203410,2.261250e+06


## **Let's plot recommended locations on map of London with current market prices**

In [22]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [23]:
# create map of Manhattan using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_pref_loc['Latitude'], df_pref_loc['Longitude'], df_pref_loc['Avg_Price'], df_pref_loc['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = '******' # your Foursquare ID
CLIENT_SECRET = '*******' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Let's explore the first neighborhood in our dataframe.

## 2. Explore the preferred location for different venues 

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each location and create a new dataframe called *location_venues* and display it.

In [28]:
# type your answer here
location_venues = getNearbyVenues(names=df_pref_loc['Street'],
                                   latitudes=df_pref_loc['Latitude'],
                                   longitudes=df_pref_loc['Longitude']
                                  )

DULWICH WOOD AVENUE
SOUTH HILL PARK
TEIGNMOUTH ROAD
BURNSALL STREET
FORDWYCH ROAD
PORTEN ROAD
ALBERT BRIDGE ROAD
EDITH VILLAS
WESTBOURNE GROVE
LADBROKE ROAD
PORTOBELLO ROAD
STRATHRAY GARDENS
WESTMORELAND ROAD
PRIORY ROAD
LEWIS CUBITT SQUARE
QUEENSMERE ROAD
RIDGWAY PLACE
DORIA ROAD
SAVERNAKE ROAD
MARLBOROUGH HILL
ELLERBY STREET
CHALCOT GARDENS
EDGE STREET
MULTON ROAD
CORNWALL TERRACE MEWS
JOHN ISLIP STREET
COURT LANE GARDENS
FRANK DIXON WAY
BUSHNELL ROAD
THURLOE PLACE MEWS
BELSIZE LANE
RATHBONE PLACE
GRANARD ROAD
ROYSTON ROAD
RADSTOCK STREET
LANCHESTER ROAD
CIRCUS ROAD WEST
TIERNEY LANE
ROTHBURY ROAD


In [29]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,DULWICH WOOD AVENUE,51.425586,-0.082416,Beer Rebellion,51.424580,-0.083425,Bar
1,DULWICH WOOD AVENUE,51.425586,-0.082416,The Indian Dining Club,51.427795,-0.086488,Indian Restaurant
2,DULWICH WOOD AVENUE,51.425586,-0.082416,The Paxton,51.427880,-0.086168,Pub
3,DULWICH WOOD AVENUE,51.425586,-0.082416,Gipsy Hill Railway Station (GIP),51.424530,-0.083959,Train Station
4,DULWICH WOOD AVENUE,51.425586,-0.082416,Brown & Green @ The Station,51.424425,-0.083836,Breakfast Spot
5,DULWICH WOOD AVENUE,51.425586,-0.082416,Manuel's Restaurant and Bar,51.427591,-0.086131,Italian Restaurant
6,SOUTH HILL PARK,51.557134,-0.164343,Daunt Books Hampstead,51.555513,-0.166277,Bookstore
7,SOUTH HILL PARK,51.557134,-0.164343,Hampstead Heath Ponds,51.559300,-0.165973,Lake
8,SOUTH HILL PARK,51.557134,-0.164343,Hampstead Heath,51.559622,-0.164921,Park
9,SOUTH HILL PARK,51.557134,-0.164343,Paradise,51.555476,-0.166312,Indian Restaurant


In [30]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBERT BRIDGE ROAD,17,17,17,17,17,17
BELSIZE LANE,25,25,25,25,25,25
BURNSALL STREET,74,74,74,74,74,74
BUSHNELL ROAD,4,4,4,4,4,4
CHALCOT GARDENS,29,29,29,29,29,29
CIRCUS ROAD WEST,23,23,23,23,23,23
CORNWALL TERRACE MEWS,85,85,85,85,85,85
COURT LANE GARDENS,12,12,12,12,12,12
DORIA ROAD,60,60,60,60,60,60


#### Let's explore the first neighborhood in our dataframe.

In [33]:
def getNearbyFacility(names, latitudes, longitudes, radius=5000):
    
    facility_list=[]
    
    radius = 5000
    LIMIT = 100
    categories = '4bf58dd8d48988d196941735,58daa1558bbb0b01f18ec1f7,4bf58dd8d48988d13d941735,4f4533804b9074f6e4fb0105,4bf58dd8d48988d118951735'
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        categories,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)

        # make the GET request
        results = requests.get(url).json()['response']

        # return only relevant information for each nearby venue
        facility_list.append([(
            name,
            facility['name'],
            facility['categories'][0]['name'],
            facility['location']['distance'], 
            facility['location']['lat'],
            facility['location']['lng']) for facility in results['venues']])

    nearby_facility = pd.DataFrame([item for f_list in facility_list for item in f_list])
    nearby_facility.columns = ['Street Name',
                  'Facility Name',             
                  'Facility Category', 
                  'Distance', 
                  'Facility Latitude', 
                  'Facility Longitude']
    
    return(nearby_facility)

In [34]:
location_facility = getNearbyFacility(names=df_pref_loc['Street'],
                                      latitudes=df_pref_loc['Latitude'],
                                      longitudes=df_pref_loc['Longitude']
                                  )

DULWICH WOOD AVENUE
SOUTH HILL PARK
TEIGNMOUTH ROAD
BURNSALL STREET
FORDWYCH ROAD
PORTEN ROAD
ALBERT BRIDGE ROAD
EDITH VILLAS
WESTBOURNE GROVE
LADBROKE ROAD
PORTOBELLO ROAD
STRATHRAY GARDENS
WESTMORELAND ROAD
PRIORY ROAD
LEWIS CUBITT SQUARE
QUEENSMERE ROAD
RIDGWAY PLACE
DORIA ROAD
SAVERNAKE ROAD
MARLBOROUGH HILL
ELLERBY STREET
CHALCOT GARDENS
EDGE STREET
MULTON ROAD
CORNWALL TERRACE MEWS
JOHN ISLIP STREET
COURT LANE GARDENS
FRANK DIXON WAY
BUSHNELL ROAD
THURLOE PLACE MEWS
BELSIZE LANE
RATHBONE PLACE
GRANARD ROAD
ROYSTON ROAD
RADSTOCK STREET
LANCHESTER ROAD
CIRCUS ROAD WEST
TIERNEY LANE
ROTHBURY ROAD


In [35]:
location_facility

,Street Name,Facility Name,Facility Category,Distance,Facility Latitude,Facility Longitude
0,DULWICH WOOD AVENUE,Tesco,Grocery Store,1311,51.436213,-0.090561
1,DULWICH WOOD AVENUE,Tesco,Supermarket,3911,51.397043,-0.049559
2,DULWICH WOOD AVENUE,M&S Simply Food,Grocery Store,4513,51.389183,-0.111039
3,DULWICH WOOD AVENUE,Tesco,Grocery Store,3677,51.445271,-0.124966
4,DULWICH WOOD AVENUE,Tesco,Grocery Store,5315,51.438880,-0.155979
5,DULWICH WOOD AVENUE,Sainsbury's Local,Grocery Store,3749,51.446901,-0.124255
6,DULWICH WOOD AVENUE,telferscot primary school,Elementary School,4250,51.442807,-0.137087
7,DULWICH WOOD AVENUE,M&S Streatham Hill Foodhall,Grocery Store,3419,51.439372,-0.126443
8,DULWICH WOOD AVENUE,The Co-operative Food,Grocery Store,2306,51.440017,-0.106273
9,DULWICH WOOD AVENUE,Tesco,Supermarket,3216,51.398649,-0.099187


In [37]:
location_facility.groupby('Street Name').count()

,Facility Name,Facility Category,Distance,Facility Latitude,Facility Longitude
Street Name,,,,,
ALBERT BRIDGE ROAD,50,50,50,50,50
BELSIZE LANE,50,50,50,50,50
BURNSALL STREET,50,50,50,50,50
BUSHNELL ROAD,50,50,50,50,50
CHALCOT GARDENS,50,50,50,50,50
CIRCUS ROAD WEST,50,50,50,50,50
CORNWALL TERRACE MEWS,50,50,50,50,50
COURT LANE GARDENS,50,50,50,50,50
DORIA ROAD,50,50,50,50,50
